In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle as pk
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression,LogisticRegression,Lasso,Ridge
from sklearn.preprocessing import StandardScaler,QuantileTransformer,PolynomialFeatures,KBinsDiscretizer,OneHotEncoder
import sys
from tqdm.notebook import tqdm

%matplotlib inline

# Load data

In [ ]:
baseline = "alert"
assert baseline in ["alert","admission"]

In [ ]:
# load data
with open(f"/edata/clarity_extracts/processed_features/outcome_analysis_features_{baseline}.pkl",'rb') as f:
    data = pk.load(f)
    
# rename some features
data.rename({'time_of_discharge':'discharge_tsp','sex':'gender'},axis=1,inplace=True)
print(data.shape)

In [ ]:
# Load alert confirmation data and merge with features
with open(f"/edata/clarity_extracts/processed_features/confirmations.pkl",'rb') as f:
    confirmations = pk.load(f)
data = data.merge(confirmations,how='left')

# Load comorbidity data and merge with features
with open(f"/edata/clarity_extracts/processed_features/mimics.pkl",'rb') as f:
    mimics = pk.load(f)    
data = data.merge(mimics,how='left')

In [ ]:
# Get the unique hospital names
hospitals = data.hospital.unique()

In [ ]:
# Precalc some temporal features
data["hours_admit_to_discharge"] = (data.discharge_tsp - data.admission_tsp)/pd.Timedelta(1,'h')
data["hours_alert_to_discharge"] = (data.discharge_tsp - data.alert_tsp)/pd.Timedelta(1,'h')

data["hours_admission_to_abx"] = data.hours_admit_to_abx
data['hours_admission_to_alert'] = (data.alert_tsp - data.admission_tsp)/pd.Timedelta(1,'h')

In [ ]:
# Pre compute time to confirmation
data['hours_alert_to_confirmation'] = (pd.to_datetime(data.confirmed_tsp,utc=True) - data.alert_tsp)/pd.Timedelta(1,'h')
data['confirmed_in_3_hrs'] = 1*(data.hours_alert_to_confirmation <= 3)

In [ ]:
# Create hospital indicators
ds_cols = []
for h in ["HCGH","BMC","JHH","SH","SMH"]:
    data[h.lower()] = 1*(data.hospital == h)
    ds_cols.append(h.lower())

In [ ]:
# Create post-covid indicator
data["post_covid"] = 1
data.loc[data.admission_tsp < pd.Timestamp(year=2020,month=4,day=1,tz='UTC'),"post_covid"] = 0

In [ ]:
# Get previous admissions (used for sensitivity analysis)
pat_enc = pd.read_csv('/edata/clarity_extracts/cdm/pat_enc.csv')
data = data.merge(pat_enc[['enc_id','pat_id']],how='left')
data.sort_values('admission_tsp',inplace=True)
data['prev_admission_tsp'] = data.groupby('pat_id').admission_tsp.shift(periods=1)
data['prev_admission'] = 1*(~data.prev_admission_tsp.isna())

In [ ]:
# Create post deployment mask
deploy_dates = {
    "HCGH":pd.Timestamp(year=2018,month=4,day=1,tz="UTC"),
    "BMC":pd.Timestamp(year=2019,month=2,day=27,tz="UTC"),
    "SH":pd.Timestamp(year=2018,month=10,day=10,tz="UTC"),
    "JHH":pd.Timestamp(year=2019,month=4,day=16,tz="UTC"),
    "SMH":pd.Timestamp(year=2019,month=5,day=15,tz="UTC")
}
post_mask = (data.hospital == "HCGH") & (data.admission_tsp >= deploy_dates['HCGH'])
post_mask = post_mask | ((data.hospital == "BMC") & (data.admission_tsp >= deploy_dates['BMC']))
post_mask = post_mask | ((data.hospital == "SH") & (data.admission_tsp >= deploy_dates['SH']))
post_mask = post_mask | ((data.hospital == "JHH") & (data.admission_tsp >= deploy_dates['JHH']))
post_mask = post_mask | ((data.hospital == "SMH") & (data.admission_tsp >= deploy_dates['SMH']))

In [ ]:
# Utility function for processing features
def get_features(data,filt,tta_col,feature_cols,ftype='mean',normalize=True):
    data_trews = data[filt]
    N = data_trews.shape[0]
    print("N = %d"%N)


    T = data_trews[tta_col].values.reshape((N,1))
    # S = data_trews["septic_shock"].values.reshape((N,1))
    X = data_trews[feature_cols].values.copy()
    age = data_trews["age"].values

    X_aug = []
    feature_cols_aug = []
    for c in range(X.shape[1]):
        nan_mask = np.isnan(X[:,c])
        if np.all((X[~nan_mask,c] == 0)|(X[~nan_mask,c]==1)):
            X[nan_mask,c] = 0
        else:
            mu = X[~nan_mask,c].mean()
            X[nan_mask,c] = mu

        if feature_cols[c].endswith("_histogram"):
            fid = feature_cols[c][:-10]
            fid_col = fid + f"_{ftype}" if fid != "age" else "age"
            
            X_c_aug = OneHotEncoder(sparse=False).fit_transform(X[:,c:c+1])
            feat = data_trews[fid_col].values.reshape((N,1))
            mu = np.nanmean(feat)
            if fid != 'gcs':
                feat[np.isnan(feat)] = mu
            else:
                feat[np.isnan(feat)] = 15
            if normalize:
                feat = (feat - mu)/np.std(feat)
            X_aug.append(X_c_aug * feat)
            feature_cols_aug += ["%s_%d"%(feature_cols[c],v) for v in range(X_c_aug.shape[1])]
            
        else:
            feat = X[:,c:c+1].copy()
            binary_feature = np.all((X[~nan_mask,c] == 0)|(X[~nan_mask,c]==1))
            if (not binary_feature) and normalize:
                feat = (feat - mu)/np.std(feat)
            X_aug.append(feat)
            feature_cols_aug.append(feature_cols[c])
    
    X = np.hstack([np.ones((N,1))] + X_aug).copy()
    feature_cols_aug.insert(0,"const")
    print("n_features = %d"%X.shape[1])
    Y = 1*((data_trews.death_in_hospital.values==1))
    
    return X,T,Y,feature_cols_aug


# Features

In [ ]:
# Feature columns
feature_cols = [
    "sofa_resp",
    "sofa_card",
    "sofa_kidn",
    "sofa_live",
    "sofa_nerv",
    "sofa_coag",
    "apache2",
    "cci",
    "gender",
    "dementia", # Harder to recognize sepsis
    "diabetes_wo_comp", # mild immumocompromised
    "diabetes_w_comp", # mild immumocompromised
    "malignancy", # immunocompromised
    "met_solid_tumor", # immunocompromised
    "abnormal_gcs",
    "lactate_ind",
    "age_histogram",
    "bp_sys_histogram",
    "heart_rate_histogram",
    "resp_rate_histogram",
    "wbc_histogram",
    'temperature_histogram',
    'gcs_histogram',
    'vasopressors',
    'mech_vent',
    'post_covid'
                ]

aux_features = [
    'cci_met_solid_tumor_prob_poa',
    'esrd_prob_poa',
    'cci_chf_prob_poa',
    'acute_liver_disease_prob_poa',
    'gi_bleed_prob_poa',
    'copd_prob_poa',
    'trauma_admit'
               ]

final_diag_features = [
    'cci_met_solid_tumor_final_diag',
    'esrd_final_diag',
    'cci_chf_final_diag',
    'acute_liver_disease_final_diag',
    'gi_bleed_final_diag',
    'copd_final_diag',
    'trauma_admit'
               ]

hospitals_in_data = np.char.lower(data.hospital.unique().astype(str))
feature_cols = feature_cols + sorted(list(hospitals_in_data))[1:]

print(feature_cols)

# Train risk model
Using retrospective data, train a model that predicts a patients increase in mortality risk with and without prompt antibiotic therapy

## Load retrospective data

In [ ]:
with open(f"/edata/clarity_extracts/processed_features/retrospective_outcome_analysis_features_{baseline}.pkl",'rb') as f:
    retro_data = pk.load(f)
print(retro_data.shape)

In [ ]:
retro_data["hours_admit_to_discharge"] = (retro_data.discharge_tsp - retro_data.admission_tsp)/pd.Timedelta(1,'h')
retro_data["hours_alert_to_discharge"] = (retro_data.discharge_tsp - retro_data.alert_tsp)/pd.Timedelta(1,'h')

retro_data["hours_admission_to_abx"] = retro_data.hours_admit_to_abx
retro_data['hours_admission_to_alert'] = (retro_data.alert_tsp - retro_data.admission_tsp)/pd.Timedelta(1,'h')

retro_data['post_covid'] = 0

In [ ]:
ds_cols = []
for h in ["HCGH","BMC","JHH","SH","SMH"]:
    retro_data[h.lower()] = 1*(retro_data.hospital == h)
    ds_cols.append(h.lower())

## Define retrospective inclusion criteria

In [ ]:
retro_filt = (retro_data["admission_tsp"] >= pd.Timestamp(year=2016,month=1,day=1,tz='UTC'))
retro_filt = retro_filt & (retro_data["admission_tsp"] < pd.Timestamp(year=2018,month=4,day=1,tz='UTC'))

# 1. Had an alert
retro_filt = retro_filt & (retro_data['hours_admission_to_alert'] > -1)
retro_filt = retro_filt & (retro_data["alert"] == 1)
print(f"{np.sum(retro_filt)} patients had an alert")

# 2. Had sepsis
retro_filt = retro_filt & (retro_data["esp"] == 1)
print(f"{np.sum(retro_filt)} patients had sepsis")

# 3. Received antibiotics 0 to 24 hours after alert
retro_filt = retro_filt & (retro_data.hours_alert_to_abx >  0)
print(f"{np.sum(retro_filt)} patients received abx after alert")

retro_filt = retro_filt & (retro_data.hours_alert_to_abx <= 24)
print(f"{np.sum(retro_filt)} patients received w/in 24 hrs of alert")

retro_filt = retro_filt & (retro_data["admit_unit_type"] != "icu")

print(f"{np.sum(retro_filt)} patients met all inclusion criteria")

## Get features and train retrospective risk model

In [ ]:
tta_col = "hours_alert_to_abx"
X_retro,T_retro,Y_retro,feature_cols_retro = get_features(retro_data,retro_filt,tta_col,feature_cols,ftype='first')
print(X_retro.shape)

In [ ]:
from sklearn.metrics import make_scorer
from sklearn.model_selection import KFold,ShuffleSplit,GridSearchCV

dfeats = ['cci','sofa','apache2']
feat_kbd = KBinsDiscretizer(n_bins=3,strategy='uniform',encode='onehot-dense').fit(retro_data[retro_filt][dfeats])
disc_retro = feat_kbd.transform(retro_data[retro_filt][dfeats])
n_int_features = disc_retro.shape[1]
n_features = X_retro.shape[1] + n_int_features

# Select hyperparameters based on the learned score's
# ability to stratify the risk ratio in a held out 
# dataset
def val_obj(mdl,X,y):
    N = X.shape[0]
    F = n_features
    Xf = X[:,:F]
    
    ts = [1,12]
    s = np.zeros((2,Xf.shape[0]))
    for t in range(2):
        T_test = t*np.ones((N,1))
        XT_test = np.hstack((Xf,Xf*T_test))
        s[t] = mdl.predict_proba(XT_test)[:,1]
    
    retro_scores = 1 - s[0]/s[1]
    rf = KBinsDiscretizer(3,encode='onehot-dense').fit_transform(retro_scores.reshape((N,1)))
    A = X[:,F:F+1]
    Xs = np.hstack([Xf,A*rf])
    rg_mask = rf[:,-1]==1
    XA = X[:,:F+1]
    lr = LogisticRegression(C=1000,solver='liblinear').fit(XA[rg_mask],y[rg_mask])
    
    res = []
    for a in range(2):
        X_aug = XA.copy()
        X_aug[:,-1] = a
        res.append(np.mean(lr.predict_proba(X_aug[rg_mask])[:,1]))
        
    
    return 1-(res[0]/res[1])

A_retro = 1*(T_retro > 3)

XT_retro = np.hstack((X_retro,disc_retro,A_retro*X_retro,disc_retro*A_retro))

lr = LogisticRegression(C=0.1,penalty='l2',solver='liblinear')
param_grid = {"C":[1000,100,10,1,0.1,0.01,0.001]}


kf = ShuffleSplit(20,test_size=0.25,random_state=4)
mort_model = GridSearchCV(lr,param_grid,cv=kf,scoring=val_obj).fit(XT_retro,Y_retro)
print(mort_model.best_params_)

ts = [1,12]
s = np.zeros((2,X_retro.shape[0]))
for t in range(2):
    T_test_retro = t*np.ones(T_retro.shape)
    XT_test_retro = np.hstack((X_retro,disc_retro,X_retro*T_test_retro,disc_retro*T_test_retro))
    s[t] = mort_model.predict_proba(XT_test_retro)[:,1]

retro_scores = 1 - s[0]/s[1]

In [ ]:
# Cross-validation scores
pd.DataFrame(mort_model.cv_results_)[['param_C','mean_test_score']]

In [ ]:
# Plot the distribution of risk scores
sns.displot(retro_scores)
plt.show()

# Plot APACHE II vs risk score
plt.scatter(retro_data[retro_filt].apache2,retro_scores)
plt.show()

In [ ]:
# Calculate risk groups on retrospective data
nrg = 3
kbd = KBinsDiscretizer(n_bins=nrg,encode='onehot-dense').fit(retro_scores.reshape((X_retro.shape[0],1)))
risk_features_retro = kbd.transform(retro_scores.reshape((X_retro.shape[0],1)))

# Define various inclusion criteria sets for deployment data

In [ ]:
inclusion_criteria = []

inclusion_criteria_names = []

feature_sets = []

In [ ]:
filt = post_mask

filt = filt & (data["admission_tsp"] < pd.Timestamp(year=2020,month=10,day=1,tz='UTC'))

# 1. Had an alert
filt = filt & (data["alert"] == 1)
print(f"{np.sum(filt)} patients had an alert")

# 2. Had sepsis?
filt = filt & (data["sepsis"] == 1)
print(f"{np.sum(filt)} patients had sepsis")

print(f'[{(data[filt]["admit_unit_type"] == "icu").sum()} admitted to icu]')

filt = filt & (data['hours_admission_to_alert'] > -1)
print(f"{np.sum(filt)} patients had alert post admission")
prev_sum = np.sum(filt)

# 3. Received antibiotics 0 to 24 hours after alert
filt = filt & ~(data.hours_alert_to_abx <  0)
cur_sum = np.sum(filt)
print(f"{np.sum(filt)} patients received abx after alert ({prev_sum-cur_sum} excluded)")
prev_sum = cur_sum

filt = filt & (data.hours_alert_to_abx <= 24)
cur_sum = np.sum(filt)
print(f"{np.sum(filt)} patients received w/in 24 hrs of alert ({prev_sum-cur_sum} excluded)")
prev_sum = cur_sum

filt = filt & (data["admit_unit_type"] != "icu")

cur_sum = np.sum(filt)
print(f"{np.sum(filt)} patients met all inclusion criteria ({prev_sum-cur_sum} excluded)")
prev_sum = cur_sum

inclusion_criteria.append(filt)
inclusion_criteria_names.append('all')
feature_sets.append(feature_cols)

In [ ]:
filt = post_mask

filt = filt & (data["admission_tsp"] < pd.Timestamp(year=2020,month=10,day=1,tz='UTC'))

# 1. Had an alert
filt = filt & (data["alert"] == 1)
print(f"{np.sum(filt)} patients had an alert")

# 2. Had sepsis?
filt = filt & (data["sepsis"] == 1)
print(f"{np.sum(filt)} patients had sepsis")

print(f'[{(data[filt]["admit_unit_type"] == "icu").sum()} admitted to icu]')

filt = filt & (data['hours_admission_to_alert'] > -1)
print(f"{np.sum(filt)} patients had alert post admission")
prev_sum = np.sum(filt)

# 3. Received antibiotics 0 to 24 hours after alert
filt = filt & ~(data.hours_alert_to_abx <  0)
cur_sum = np.sum(filt)
print(f"{np.sum(filt)} patients received abx after alert ({prev_sum-cur_sum} excluded)")
prev_sum = cur_sum

filt = filt & (data.hours_alert_to_abx <= 24)
cur_sum = np.sum(filt)
print(f"{np.sum(filt)} patients received w/in 24 hrs of alert ({prev_sum-cur_sum} excluded)")
prev_sum = cur_sum

filt = filt & (data["admit_unit_type"] != "icu")

cur_sum = np.sum(filt)
print(f"{np.sum(filt)} patients met all inclusion criteria ({prev_sum-cur_sum} excluded)")
prev_sum = cur_sum

inclusion_criteria.append(filt)
inclusion_criteria_names.append('unique_patients')
feature_sets.append(feature_cols + aux_features)

In [ ]:
filt = post_mask

# 1. Pre-COVID
filt = filt & (data["admission_tsp"] < pd.Timestamp(year=2020,month=4,day=1,tz='UTC'))

# 2. Had an alert
filt = filt & (data['hours_admission_to_alert'] > -1)
filt = filt & (data["alert"] == 1)
print(f"{np.sum(filt)} patients had an alert")

# 3. Had sepsis?
filt = filt & (data["sepsis"] == 1)
print(f"{np.sum(filt)} patients had sepsis")

# 4. Received antibiotics 0 to 24 hours after alert
filt = filt & (data.hours_alert_to_abx >  0)
print(f"{np.sum(filt)} patients received abx after alert")

filt = filt & (data.hours_alert_to_abx <= 24)
print(f"{np.sum(filt)} patients received w/in 24 hrs of alert")

filt = filt & (data["admit_unit_type"] != "icu")

print(f"{np.sum(filt)} patients met all inclusion criteria")

inclusion_criteria.append(filt)
inclusion_criteria_names.append('pre_covid')
feature_sets.append(feature_cols + aux_features)

In [ ]:
filt = post_mask

filt = filt & (data["admission_tsp"] < pd.Timestamp(year=2020,month=10,day=1,tz='UTC'))

# 1. Had an alert
filt = filt & (data["alert"] == 1)
print(f"{np.sum(filt)} patients had an alert")

# 2. Had sepsis?
filt = filt & (data["sepsis"] == 1)
print(f"{np.sum(filt)} patients had sepsis")

print(f'[{(data[filt]["admit_unit_type"] == "icu").sum()} admitted to icu]')

filt = filt & (data['hours_admission_to_alert'] > -1)
print(f"{np.sum(filt)} patients had alert post admission")
prev_sum = np.sum(filt)

# 3. Received antibiotics 0 to 24 hours after alert
filt = filt & ~(data.hours_alert_to_abx <  0)
cur_sum = np.sum(filt)
print(f"{np.sum(filt)} patients received abx after alert ({prev_sum-cur_sum} excluded)")
prev_sum = cur_sum

filt = filt & (data.hours_alert_to_abx <= 24)
cur_sum = np.sum(filt)
print(f"{np.sum(filt)} patients received w/in 24 hrs of alert ({prev_sum-cur_sum} excluded)")
prev_sum = cur_sum

filt = filt & (data["admit_unit_type"] != "icu")

cur_sum = np.sum(filt)
print(f"{np.sum(filt)} patients met all inclusion criteria ({prev_sum-cur_sum} excluded)")
prev_sum = cur_sum

inclusion_criteria.append(filt)
inclusion_criteria_names.append('all_w_mimics')
feature_sets.append(feature_cols + aux_features)

In [ ]:
filt = post_mask

filt = filt & (data["admission_tsp"] < pd.Timestamp(year=2020,month=10,day=1,tz='UTC'))

# 1. Had an alert
filt = filt & (data["alert"] == 1)
print(f"{np.sum(filt)} patients had an alert")

# 2. Had sepsis?
filt = filt & (data["sepsis"] == 1)
print(f"{np.sum(filt)} patients had sepsis")

print(f'[{(data[filt]["admit_unit_type"] == "icu").sum()} admitted to icu]')

filt = filt & (data['hours_admission_to_alert'] > -1)
print(f"{np.sum(filt)} patients had alert post admission")
prev_sum = np.sum(filt)

# 3. Received antibiotics 0 to 24 hours after alert
filt = filt & ~(data.hours_alert_to_abx <  0)
cur_sum = np.sum(filt)
print(f"{np.sum(filt)} patients received abx after alert ({prev_sum-cur_sum} excluded)")
prev_sum = cur_sum

filt = filt & (data.hours_alert_to_abx <= 24)
cur_sum = np.sum(filt)
print(f"{np.sum(filt)} patients received w/in 24 hrs of alert ({prev_sum-cur_sum} excluded)")
prev_sum = cur_sum

filt = filt & (data["admit_unit_type"] != "icu")

cur_sum = np.sum(filt)
print(f"{np.sum(filt)} patients met all inclusion criteria ({prev_sum-cur_sum} excluded)")
prev_sum = cur_sum

inclusion_criteria.append(filt)
inclusion_criteria_names.append('all_w_final_diags')
feature_sets.append(feature_cols + final_diag_features)

In [ ]:
filt = post_mask

filt = filt & (data["admission_tsp"] < pd.Timestamp(year=2020,month=10,day=1,tz='UTC'))

# 1. Had an alert
filt = filt & (data["alert"] == 1)
print(f"{np.sum(filt)} patients had an alert")

# 2. Had sepsis?
filt = filt & (data["sepsis"] == 1)
print(f"{np.sum(filt)} patients had sepsis")

print(f'[{(data[filt]["admit_unit_type"] == "icu").sum()} admitted to icu]')

filt = filt & (data['hours_admission_to_alert'] > -1)
print(f"{np.sum(filt)} patients had alert post admission")
prev_sum = np.sum(filt)

# 3. Received antibiotics 0 to 24 hours after alert
filt = filt & ~(data.hours_alert_to_abx <  0)
cur_sum = np.sum(filt)
print(f"{np.sum(filt)} patients received abx after alert ({prev_sum-cur_sum} excluded)")
prev_sum = cur_sum

filt = filt & (data.hours_alert_to_abx <= 24)
cur_sum = np.sum(filt)
print(f"{np.sum(filt)} patients received w/in 24 hrs of alert ({prev_sum-cur_sum} excluded)")
prev_sum = cur_sum

filt = filt & (data["admit_unit_type"] != "icu")


filt = filt & ~((data.hours_alert_to_abx <= 3) & (~(data.hours_alert_to_confirmation <= 3)))

cur_sum = np.sum(filt)
print(f"{np.sum(filt)} patients met all inclusion criteria ({prev_sum-cur_sum} excluded)")
prev_sum = cur_sum

inclusion_criteria.append(filt)
inclusion_criteria_names.append('excludes_treated_not_confirmed')
feature_sets.append(feature_cols)

# Marginal analysis

In [ ]:
################################################
# Set treatment='confirmed' to use time to confirmation
# as the exposure variable and treatment='abx' to 
# use time to antibiotics as the exposure variable
################################################
treatment = "confirmed"

tta_col = "hours_alert_to_abx"

# number of bootstrap replicates
n_bs_samples = 5001

res_dicts = {}
for filt,filt_name,feats in zip(inclusion_criteria,inclusion_criteria_names,feature_sets):
    print(f'**** Cohort: {filt_name} ****')
    X,T,Y,feature_cols_aug = get_features(data,filt,tta_col,feats,ftype='first')
    print()
    
    res_dict = {}
    res_dict['X'] = X
    res_dict['Y'] = Y
    res_dict['T'] = T
    res_dict['filter'] = filt
    res_dict['data'] = data
    
    
    # SOFA 72
    if treatment == "abx":
        A = 1*(T > 3)
    elif treatment == "confirmed":
        A = 1-1*data[filt].confirmed_in_3_hrs.values.reshape((X.shape[0],1))
    XA = np.hstack((X,A))
    print(f'SOFA @ 72:')
    for i,t in enumerate([72]):
        S = data[filt][f'sofa_{t}'].values
        t_mask = ~(data[filt].discharge_type.isin(['censored','hospice']))
        fmask = ~np.all(XA[t_mask]==0,axis=0)
        disc_ols_res = sm.OLS(S[t_mask],XA[t_mask][:,fmask]).fit(cov_type='HC1')
        pe = disc_ols_res.params[-1]
        ci = disc_ols_res.conf_int()[-1]
        p = disc_ols_res.pvalues[-1]
        print(f"{pe:0.2f} [{ci[0]:.2f} - {ci[1]:.2f}] p={p:0.3f}")
        res_dict['sofa_at_72'] = {'point_estimate':pe,'confidence_interval':ci,'p_value':p}
        
    print()
    
    print(f'Length of stay:')
    los_mask = Y==0
    LOS = (data[filt].hours_alert_to_discharge.values)
    fmask = ~np.all(XA[los_mask]==0,axis=0)
    disc_qr_res = sm.QuantReg(LOS[los_mask],XA[los_mask][:,fmask]).fit(q=0.5,max_iter=5000)
    pe = disc_qr_res.params[-1]
    ci = disc_qr_res.conf_int()[-1]
    p = disc_qr_res.pvalues[-1]
    print(f"{pe:0.2f} [{ci[0]:.2f} - {ci[1]:.2f}] p={p:0.3f}")
    
    res_dict['length_of_stay'] = {'point_estimate':pe,'confidence_interval':ci,'p_value':p}
    print()
    
    bs_res = np.zeros(n_bs_samples)
    ipw_res = np.zeros((n_bs_samples,2))
    np.random.seed(5)
    preds = np.zeros((n_bs_samples,2,Y.shape[0]))
    XA = np.hstack((X,A))
    for s in tqdm(list(range(n_bs_samples)),leave=False):
        if s == 0:
            idxs = np.arange(Y.shape[0])
        else:
            idxs = np.random.choice(Y.shape[0],Y.shape[0],replace=True)
        XA_s = XA[idxs]
        Y_s = Y[idxs]
        lr = LogisticRegression(C=1000,solver='liblinear').fit(XA_s,Y_s)
        preds_s = np.zeros((2,Y.shape[0]))
        for a in range(2):
            XA_s_int = XA_s.copy()
            XA_s_int[:,-1] = a
            preds[s,a] = lr.predict_proba(XA_s_int)[:,1]
        bs_res[s] = 1 - np.mean(preds[s,0])/np.mean(preds[s,1])

    print('Mortality:')
    print((np.mean(preds[0,1]),np.mean(preds[0,0])))
    rds = np.mean(preds[:,1] - preds[:,0],axis=1)
    rd_ci = np.percentile(rds[1:],[2.5,97.5])
    rd_p = 2*min(np.mean(rds[1:] > 0),np.mean(rds[1:] < 0))
    print(f"RD: {100*rds[0]:.2f}% [{100*rd_ci[0]:.2f}% - {100*rd_ci[1]:.2f}%] p={rd_p:.3f}")
    rrs = 1-np.mean(preds[:,0],axis=1)/np.mean(preds[:,1],axis=1)
    rr_ci = np.percentile(rrs[1:],[2.5,97.5])
    rr_p = 2*min(np.mean(rrs[1:] > 0),np.mean(rrs[1:] < 0))
    print(f"Rel red: {100*rrs[0]:.2f}% [{100*rr_ci[0]:.2f}% - {100*rr_ci[1]:.2f}%] p={rr_p:.3f}")
    print()
    
    res_dict['mortality'] = {'bootstrap_point_estimates':preds.mean(-1)}
    
    res_dicts[filt_name] = res_dict

In [ ]:
tta_col = "hours_alert_to_abx"

# number of bootstrap replicates
n_bs_samples = 5001

for filt,filt_name,feats in zip(inclusion_criteria,inclusion_criteria_names,feature_sets):
    print(f'**** Cohort: {filt_name} ****')
    X,T,Y,feature_cols_aug = get_features(data,filt,tta_col,feats,ftype='first')
    print()
    
    # SOFA 72
    XT = np.hstack((X,T))
    print(f'Per hour mortality:')
    
    res_dicts[filt_name]['per_hour_mortality'] = {}
    for i,t in enumerate([6,12,24]):
        print(f'** tmax = {t} **')
        res = np.zeros(n_bs_samples)
        for s in tqdm(list(range(n_bs_samples)),leave=False):
            if s == 0:
                idxs = np.arange(X.shape[0])
            else:
                idxs = np.random.choice(X.shape[0],X.shape[0],replace=True)
                
            XT_s = XT[idxs]
            Y_s = Y[idxs]
            T_s = T[idxs]
            t_mask = T_s[:,0] <= t
            fmask = ~np.all(XT_s[t_mask]==0,axis=0)
            logit_res = LogisticRegression(C=1000,solver='liblinear').fit(XT_s[t_mask][:,fmask],Y_s[t_mask])
            res[s] = np.exp(logit_res.coef_[0,-1])
            
        pe = res[0]
        ci = np.percentile(res[1:],[2.5,97.5])
        p = 2*min(np.mean(res[1:] > 1),np.mean(res[1:] < 1))
        print(f"{pe:0.2f} [{ci[0]:.2f} - {ci[1]:.2f}] p={p:0.3f}")
        
        res_dicts[filt_name]['per_hour_mortality'][f'bootstrap_point_estimates_{t}'] = res
        
    print()

### Save out results

In [ ]:
for filt_name in inclusion_criteria_names:
    with open(f'/home/radams47/trews_deployment_analysis/output/{filt_name}_{treatment}_marginal_results.pkl','wb') as f:
        pk.dump(res_dicts[filt_name],f)

### Unadjusted outcomes

In [ ]:
for filt,filt_name,feats in zip(inclusion_criteria,inclusion_criteria_names,feature_sets):
    print(f'**** Cohort: {filt_name} ****')
    X,T,Y,feature_cols_aug = get_features(data,filt,tta_col,feats,ftype='first')
    print()
    
    # SOFA 72
    XT = np.hstack((X,T))
    if treatment == "abx":
        A = T[:,0] <= 3
#     A = 1*(~((pd.to_datetime(data[filt].eval_tsp) - data[filt].alert_tsp) <= pd.Timedelta(3,'h'))).values.reshape((X.shape[0],1))
    elif treatment == "confirmed":
        A = data[filt].confirmed_in_3_hrs.values==1
    print('N')
    print(f'{np.sum(A==1)} | {np.sum(A==0)}')
    print()
    
    print('Mortality')
    print(f'{np.sum(Y[(A==1)])} ({np.mean(Y[(A==1)]):.1%}) | {np.sum(Y[(A==0)])} ({np.mean(Y[(A==0)]):.1%})')
    print()

    print('SOFA @ 72')
    t_mask = ~(data[filt].discharge_type.isin(['censored','hospice']))
    S = data[filt][f'sofa_72'].values - data[filt][f'sofa'].values
    print(f'{np.mean(S[t_mask&(A==1)]):.1f} ± {np.std(S[t_mask&(A==1)]):.1f} | {np.mean(S[t_mask&(A==0)]):.1f} ± {np.std(S[t_mask&(A==0)]):.1f}')
    print()

    print('LOS')
    los_mask = Y==0
    LOS = (data[filt].hours_alert_to_discharge.values)
    m1 = np.median(LOS[los_mask&(A==1)])
    lq1 = np.quantile(LOS[los_mask&(A==1)],0.25)
    uq1 = np.quantile(LOS[los_mask&(A==1)],0.75)
    m0 = np.median(LOS[los_mask&(A==0)])
    lq0 = np.quantile(LOS[los_mask&(A==0)],0.25)
    uq0 = np.quantile(LOS[los_mask&(A==0)],0.75)
    print(f'{m1:.0f} ({lq1:.0f} - {uq1:.0f}) | {m0:.0f} ({lq0:.0f} - {uq0:.0f})')
    print()

### Rerun using IPTW

In [ ]:
import robustats as rs

In [ ]:
treatment = "confirmed"
tta_col = "hours_alert_to_abx"
n_bs_samples = 11

iptw_results = {}
for filt,filt_name,feats in zip(inclusion_criteria,inclusion_criteria_names,feature_sets):
    print(f'**** Cohort: {filt_name} ****')
    X,T,Y,feature_cols_aug = get_features(data,filt,tta_col,feats,ftype='first')
    print()
    
    # SOFA 72
    if treatment == "abx":
        A = 1*(T > 3)
    elif treatment == "confirmed":
        A = 1-1*data[filt].confirmed_in_3_hrs.values.reshape((X.shape[0],1))
    XA = np.hstack((X,A))
    
    LOS = (data[filt].hours_alert_to_discharge.values)
    los_mask = Y==0
    
    S = data[filt][f'sofa_72'].values
    s_mask = ~(data[filt].discharge_type.isin(['censored','hospice'])).values
    
    res_d = {
        'sofa':np.zeros((n_bs_samples,2)),
        'los':np.zeros((n_bs_samples,2)),
        'mortality':np.zeros((n_bs_samples,2)),
    }
    
    for s in tqdm(list(range(n_bs_samples)),leave=False):
        if s == 0:
            idxs = np.arange(Y.shape[0])
        else:
            idxs = np.random.choice(Y.shape[0],Y.shape[0],replace=True)
        X_s = X[idxs]
        A_s = A[idxs][:,0]
        Y_s = Y[idxs]
        S_s = S[idxs]
        LOS_s = LOS[idxs]
        s_mask_s = s_mask[idxs]
        los_mask_s = los_mask[idxs]
        
        treatment_model = LogisticRegression(C=1000,solver='liblinear').fit(X_s,A_s)
        
        pa = treatment_model.predict_proba(X_s)[:,1]
        bnds = np.quantile(pa,[0.01,0.99])
        pa = np.clip(pa,a_min=bnds[0],a_max=bnds[1])
        
        ey0 = np.sum(Y_s*(1-A_s)/(1-pa))/np.sum((1-A_s)/(1-pa))
        ey1 = np.sum(Y_s*A_s/pa)/np.sum(A_s/pa)
        
        res_d['mortality'][s,0] = ey0
        res_d['mortality'][s,1] = ey1
        
        es0 = np.sum(S_s*(1-A_s)*s_mask_s/(1-pa))/np.sum((1-A_s)*s_mask_s/(1-pa))
        es1 = np.sum(S_s*A_s*s_mask_s/pa)/np.sum(A_s*s_mask_s/pa)
        
        res_d['sofa'][s,0] = es0
        res_d['sofa'][s,1] = es1
        
        mlos0 = rs.weighted_median(LOS_s[los_mask_s & (A_s==0)],1/(1-pa[los_mask_s & (A_s==0)]))
        mlos1 = rs.weighted_median(LOS_s[los_mask_s & (A_s==1)],1/(pa[los_mask_s & (A_s==1)]))
    
        res_d['los'][s,0] = mlos0
        res_d['los'][s,1] = mlos1
        
    rds = res_d["sofa"][:,1] - res_d["sofa"][:,0]
    ci = np.percentile(rds,[2.5,97.5])
    print(f'IPTW SOFA: {rds[0]:.2f} [{ci[0]:.2f}, {ci[1]:.2f}]')
    
    rds = res_d["los"][:,1] - res_d["los"][:,0]
    ci = np.percentile(rds,[2.5,97.5])
    print(f'IPTW LOS: {rds[0]:.1f} [{ci[0]:.1f}, {ci[1]:.1f}]')
    
    rds = res_d["mortality"][:,1] - res_d["mortality"][:,0]
    ci = np.percentile(rds,[2.5,97.5])
    print(f'IPTW mortality: {rds[0]:.2%} [{ci[0]:.2%}, {ci[1]:.2%}]')
    
    iptw_results[filt_name] = res_d

### Save out IPTW results

In [ ]:
for filt_name in inclusion_criteria_names:
    with open(f'/home/radams47/trews_deployment_analysis/output/{filt_name}_{treatment}_iptw_results.pkl','wb') as f:
        pk.dump(iptw_results[filt_name],f)

# High-risk analysis
Main analysis stratified by risk group

In [ ]:
tta_col = "hours_alert_to_abx"

# number of bootstrap replicates
n_bs_samples = 5001

high_risk_res_dicts = {}

for filt,filt_name,feats in zip(inclusion_criteria,inclusion_criteria_names,feature_sets):
    print(f'**** Cohort: {filt_name} ****')
    X,T,Y,feature_cols_aug = get_features(data,filt,tta_col,feats,ftype='first')
    print()
    
    res_dict = {}
    res_dict['X'] = X
    res_dict['Y'] = Y
    res_dict['T'] = T
    res_dict['filter'] = filt
    res_dict['data'] = data
    
    Xr,_,_,_ = get_features(data,filt,tta_col,feature_cols,ftype='first')
    disc = feat_kbd.transform(data[filt][dfeats])
    s = np.zeros((2,X.shape[0]))
    for t in range(2):
        T_test = t*np.ones(T.shape)
        XT_test = np.hstack((Xr,disc,Xr*T_test,disc*T_test))
        s[t] = mort_model.predict_proba(XT_test)[:,1]

    scores = 1 - s[0]/s[1]

    risk_features = kbd.transform(scores.reshape((X.shape[0],1)))
    print(np.sum(risk_features,axis=0))
    print()
    
    res_dict['risk_scores'] = scores
    res_dict['risk_features'] = risk_features
    
    for rg in range(nrg):
        res_dict[f'risk_group_{rg}'] = {}
        
        print(f'** Risk group: {rg} **')
        rmask = risk_features[:,rg] == 1
        
        # SOFA 72
        if treatment == "abx":
            A = 1*(T > 3)
        elif treatment == "confirmed":
            A = 1-1*data[filt].confirmed_in_3_hrs.values.reshape((X.shape[0],1))
        XA = np.hstack((X,A))
        print(f'SOFA @ 72:')
        for i,t in enumerate([72]):
            S = data[filt][f'sofa_{t}'].values
            t_mask = ~(data[filt].discharge_type.isin(['censored','hospice']))
            fmask = ~np.all(XA[t_mask]==0,axis=0)
            disc_ols_res = sm.OLS(S[t_mask&rmask],XA[t_mask&rmask][:,fmask]).fit(cov_type='HC1')
            pe = disc_ols_res.params[-1]
            ci = disc_ols_res.conf_int()[-1]
            p = disc_ols_res.pvalues[-1]
            print(f"{pe:0.2f} [{ci[0]:.2f} - {ci[1]:.2f}] p={p:0.3f}")
            res_dict[f'risk_group_{rg}']['sofa_at_72'] = {'point_estimate':pe,'confidence_interval':ci,'p_value':p}
        print()

        print(f'Length of stay:')
        los_mask = Y==0
        LOS = (data[filt].hours_alert_to_discharge.values)
        fmask = ~np.all(XA[los_mask&rmask]==0,axis=0)
        disc_qr_res = sm.QuantReg(LOS[los_mask&rmask],XA[los_mask&rmask][:,fmask]).fit(q=0.5,max_iter=5000)
        pe = disc_qr_res.params[-1]
        ci = disc_qr_res.conf_int()[-1]
        p = disc_qr_res.pvalues[-1]
        print(f"{pe:0.2f} [{ci[0]:.2f} - {ci[1]:.2f}] p={p:0.3f}")
        res_dict[f'risk_group_{rg}']['length_of_stay'] = {'point_estimate':pe,'confidence_interval':ci,'p_value':p}

        bs_res = np.zeros(n_bs_samples)
        np.random.seed(5)
        preds = np.zeros((n_bs_samples,2,rmask.sum()))
        XA = np.hstack((X,A))
        for s in tqdm(list(range(n_bs_samples)),leave=False):
            if s == 0:
                idxs = np.arange(rmask.sum())
            else:
                idxs = np.random.choice(rmask.sum(),rmask.sum(),replace=True)
            XA_s = XA[rmask][idxs]
            Y_s = Y[rmask][idxs]
            lr = LogisticRegression(C=1000,solver='liblinear').fit(XA_s,Y_s)
            for a in range(2):
                XA_s_int = XA_s.copy()
                XA_s_int[:,-1] = a
                preds[s,a] = lr.predict_proba(XA_s_int)[:,1]
            bs_res[s] = 1 - np.mean(preds[s,0])/np.mean(preds[s,1])

        print('Mortality:')
        print((np.mean(preds[0,1]),np.mean(preds[0,0])))
        rds = np.mean(preds[:,1] - preds[:,0],axis=1)
        rd_ci = np.percentile(rds[1:],[2.5,97.5])
        rd_p = 2*min(np.mean(rds[1:] > 0),np.mean(rds[1:] < 0))
        print(f"RD: {100*rds[0]:.2f}% [{100*rd_ci[0]:.2f}% - {100*rd_ci[1]:.2f}%] p={rd_p:.3f}")
        rrs = 1-np.mean(preds[:,0],axis=1)/np.mean(preds[:,1],axis=1)
        rr_ci = np.percentile(rrs[1:],[2.5,97.5])
        rr_p = 2*min(np.mean(rrs[1:] > 0),np.mean(rrs[1:] < 0))
        print(f"Rel red: {100*rrs[0]:.2f}% [{100*rr_ci[0]:.2f}% - {100*rr_ci[1]:.2f}%] p={rr_p:.3f}")
        print()
        
        res_dict[f'risk_group_{rg}']['mortality'] = {'bootstrap_point_estimates':preds.mean(-1)}
    
    high_risk_res_dicts[filt_name] = res_dict

In [ ]:
tta_col = "hours_alert_to_abx"

for filt,filt_name,feats in zip(inclusion_criteria,inclusion_criteria_names,feature_sets):
    print(f'**** Cohort: {filt_name} ****')
    X,T,Y,feature_cols_aug = get_features(data,filt,tta_col,feats,ftype='first')
    print()
    
    Xr,_,_,_ = get_features(data,filt,tta_col,feature_cols,ftype='first')
    disc = feat_kbd.transform(data[filt][dfeats])
    s = np.zeros((2,X.shape[0]))
    for t in range(2):
        T_test = t*np.ones(T.shape)
        XT_test = np.hstack((Xr,disc,Xr*T_test,disc*T_test))
        s[t] = mort_model.predict_proba(XT_test)[:,1]

    scores = 1 - s[0]/s[1]

    risk_features = kbd.transform(scores.reshape((X.shape[0],1)))
    print(np.sum(risk_features,axis=0))
    print()
    for rg in range(nrg):
        high_risk_res_dicts[filt_name][f'risk_group_{rg}']['per_hour_mortality'] = {}
        
        print(f'** Risk group: {rg} **')
        rmask = risk_features[:,rg] == 1
        
        # SOFA 72
        XT = np.hstack((X,T))
        for i,t in enumerate([6,12,24]):
            
            print(f'** tmax = {t} **')
            res = np.zeros(n_bs_samples)
            for s in tqdm(list(range(n_bs_samples)),leave=False):
                if s == 0:
                    idxs = np.arange(rmask.sum())
                else:
                    idxs = np.random.choice(rmask.sum(),rmask.sum(),replace=True)

                XT_s = XT[rmask][idxs]
                Y_s = Y[rmask][idxs]
                T_s = T[rmask][idxs]
                t_mask = T_s[:,0] <= t
                fmask = ~np.all(XT_s[t_mask]==0,axis=0)
                logit_res = LogisticRegression(C=1000,solver='liblinear').fit(XT_s[t_mask][:,fmask],Y_s[t_mask])
                res[s] = np.exp(logit_res.coef_[0,-1])

            pe = res[0]
            ci = np.percentile(res[1:],[2.5,97.5])
            p = 2*min(np.mean(res[1:] > 1),np.mean(res[1:] < 1))
            print(f"{pe:0.2f} [{ci[0]:.2f} - {ci[1]:.2f}] p={p:0.3f}")
            high_risk_res_dicts[filt_name][f'risk_group_{rg}']['per_hour_mortality'][f'bootstrap_point_estimates_{t}'] = res
        print()

### Save out risk-stratified results

In [ ]:
for filt_name in inclusion_criteria_names:
    with open(f'/home/radams47/trews_deployment_analysis/output/{filt_name}_{treatment}_high_risk_results.pkl','wb') as f:
        pk.dump(high_risk_res_dicts[filt_name],f)

### Unadjusted outcomes by risk group

In [ ]:
for filt,filt_name,feats in zip(inclusion_criteria,inclusion_criteria_names,feature_sets):
    print(f'**** Cohort: {filt_name} ****')
    X,T,Y,feature_cols_aug = get_features(data,filt,tta_col,feats,ftype='first')
    print()
    
    Xr,_,_,_ = get_features(data,filt,tta_col,feature_cols,ftype='first')
    disc = feat_kbd.transform(data[filt][dfeats])
    s = np.zeros((2,X.shape[0]))
    for t in range(2):
        T_test = t*np.ones(T.shape)
        XT_test = np.hstack((Xr,disc,Xr*T_test,disc*T_test))
        s[t] = mort_model.predict_proba(XT_test)[:,1]

    scores = 1 - s[0]/s[1]

    risk_features = kbd.transform(scores.reshape((X.shape[0],1)))
    print(np.sum(risk_features,axis=0))
    print()
    for rg in range(nrg):
        print(f'** Risk group: {rg} **')
        rmask = risk_features[:,rg] == 1
        
        # SOFA 72
        XT = np.hstack((X,T))
        if treatment == "abx":
            A = T[:,0] <= 3
        elif treatment == "confirmed":
            A = data[filt].confirmed_in_3_hrs.values==1
        print('N')
        print(f'{np.sum(rmask&(A==1))} | {np.sum(rmask&(A==0))}')
        print()
        
        print('Mortality')
        print(f'{np.sum(Y[rmask&(A==1)])} ({np.mean(Y[rmask&(A==1)]):.1%}) | {np.sum(Y[rmask&(A==0)])} ({np.mean(Y[rmask&(A==0)]):.1%})')
        print()
        
        print('SOFA @ 72')
        t_mask = ~(data[filt].discharge_type.isin(['censored','hospice']))
        S = data[filt][f'sofa_72'].values - data[filt][f'sofa'].values
        print(f'{np.mean(S[rmask&t_mask&(A==1)]):.1f} ± {np.std(S[rmask&t_mask&(A==1)]):.1f} | {np.mean(S[rmask&t_mask&(A==0)]):.1f} ± {np.std(S[rmask&t_mask&(A==0)]):.1f}')
        print()
        
        print('LOS')
        los_mask = Y==0
        LOS = (data[filt].hours_alert_to_discharge.values)
        m1 = np.median(LOS[rmask&los_mask&(A==1)])
        lq1 = np.quantile(LOS[rmask&los_mask&(A==1)],0.25)
        uq1 = np.quantile(LOS[rmask&los_mask&(A==1)],0.75)
        m0 = np.median(LOS[rmask&los_mask&(A==0)])
        lq0 = np.quantile(LOS[rmask&los_mask&(A==0)],0.25)
        uq0 = np.quantile(LOS[rmask&los_mask&(A==0)],0.75)
        print(f'{m1:.0f} ({lq1:.0f} - {uq1:.0f}) | {m0:.0f} ({lq0:.0f} - {uq0:.0f})')
        print()

### Save out risk strata

In [ ]:
high_risk_res_dicts['all'].keys()

In [ ]:
sv_filt = high_risk_res_dicts['all']['filter']
rf = high_risk_res_dicts['all']['risk_features']
X = high_risk_res_dicts['all']['X']
risk_strata = data[sv_filt][['enc_id','admission_tsp','alert_tsp','discharge_tsp','first_antibiotics_tsp','first_antibiotics_order_tsp','death_in_hospital','discharge_type','sofa']].copy()
risk_strata['dataset_id'] = 77
for i,rg in enumerate(['low','mid','high']):
    risk_strata[f'{rg}_risk'] = rf[:,i]
    
for c,f in enumerate(feature_cols_aug[1:]):
    risk_strata[f] = X[:,c+1]
    
risk_strata.to_csv('/edata/clarity_extracts/processed_features/risk_strata.csv')